# DATA 601 Final Project
# Severe Thunderstorm Climatology vs Sunspot Number 2019-2020
## Python weather analysis application that compares the occurrence of severe thunderstorm activity occurring over the continental Unites States to daily international sunspot number measurements.

In [10]:
#Load modules and invoke WxData API
from __future__ import print_function, division
import numpy as np
import pandas as pd 
import sys
import requests
import xmltodict

## Data Cleaning: Define a function that builds a storm report listing for each semi-monthly period between November 2019 and April 2020.

In [11]:
#Function that builds a storm report listing for each semi-monthly period between November 2019 and April 2020.
def dataframe_strmrpt(sr_month):
    responsewx = requests.get(sr_month)
    wxdata_sr = responsewx.content
    xmlDict = xmltodict.parse(wxdata_sr) #Build storm report dictionary from XML files
    
    #Initiate lists for each storm report attribuite
    id = [] #Report ID of each severe thunderstorm event
    mag = [] #Thunderstorm wind gust magnitude in units of miles per hour (MPH)
    state = [] #US state of occurrence of severe thunderstorm event
    date = [] #Date and time of occurrence of severe thunderstorm event
    
    if xmlDict['response']['data'].get('storm-report')!=None: #Test for the existence of storm reports for the period of interest
        print("The key 'storm-report' is present.\n") 
        #If storm reports are present, build the storm report list
        for stormreports in xmlDict['response']['data']['storm-report']:
            id.append(stormreports['id'])
            mag.append(stormreports['magnitude'])
            state.append(stormreports['state'])
            date.append(stormreports['event_local_time'])

        # Reversing a list using reverse() 
        def Reverse(lst): 
            lst.reverse() 
            return lst 
        #Build Pandas series for each storm report attribuite
        sr_id = pd.Series(Reverse(id))
        sr_date = pd.Series(Reverse(date))
        sr_state = pd.Series(Reverse(state))
        sr_mag = pd.Series(Reverse(mag))
        #Build a Pandas dataframe from each individual storm report series
        #Extract maximum thunderstorm wind measurement in units of miles per hour (MPH) for each semi-monthly period
        wxdata_df = pd.DataFrame({'ID':sr_id,'Date':sr_date,'State':sr_state,'Magnitude':sr_mag})
        #Clean dataset, remove MPH label from string, save each wind report as an integer 
        wxdata_df['Magnitude'] = wxdata_df['Magnitude'].str.replace(r'\M', '')
        wxdata_df['Magnitude'] = wxdata_df['Magnitude'].str.replace(r'\E', '')
        wxdata_df['Magnitude'] = wxdata_df['Magnitude'].str.replace(r'\PH', '')
        wxdata_df['Magnitude'] = wxdata_df['Magnitude'].str.replace(r'\.00', '')
        wxdata_df['Magnitude'] = wxdata_df['Magnitude'].astype(int)
            
    else: 
        #If no storm reports exist, set analysis parameters = 0 and the dataframe = None
        print("The key 'storm-report' does not exist in the dictionary.") 
        wxdata_df = 'None'
        
    return wxdata_df #Return storm report dataframe 

In [12]:
def concat_wxdf(df_wxdata_twg,df_wxdata_mtw):
    df_wxdata = df_wxdata_twg.append(df_wxdata_mtw, ignore_index=True)
    print(df_wxdata.head())
    return df_wxdata


## API Data Collection: Invoke Requests to call the WxData Storm Reports API Builder.

In [13]:
#Invoke requests to call the WxData Storm Reports API Builder. Execute an API call for each semi-monthly period between November 2019 and April 2020.
#See https://wxdata.com/storm-reports-api-format#event
nov2019_1_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2019-11-01T00:00:00&enddate=2019-11-16T00:00:00&event=tstm+wnd+gst"
nov2019_1_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2019-11-01T00:00:00&enddate=2019-11-16T00:00:00&event=marine+tstm+wind"
nov2019_2_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2019-11-16T00:00:00&enddate=2019-12-01T00:00:00&event=tstm+wnd+gst"
nov2019_2_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2019-11-16T00:00:00&enddate=2019-12-01T00:00:00&event=tstm+wnd+gst"

df_wxdata_nov2019e_twg = dataframe_strmrpt(nov2019_1_twg)
df_wxdata_nov2019e_mtw = dataframe_strmrpt(nov2019_1_mtw)
df_wxdata_nov2019e = concat_wxdf(df_wxdata_nov2019e_twg,df_wxdata_nov2019e_mtw)
df_wxdata_nov2019e.to_csv(r'data\wxdata_nov2019e.csv')
print("wxdata_nov2019e.csv saved")

df_wxdata_nov2019l_twg = dataframe_strmrpt(nov2019_2_twg)
df_wxdata_nov2019l_mtw = dataframe_strmrpt(nov2019_2_mtw)
df_wxdata_nov2019l = concat_wxdf(df_wxdata_nov2019l_twg,df_wxdata_nov2019l_mtw)
df_wxdata_nov2019l.to_csv(r'data\wxdata_nov2019l.csv')
print("wxdata_nov2019l.csv saved")


The key 'storm-report' is present.

The key 'storm-report' is present.

                  Date      ID  Magnitude State
0  2019-11-01 01:13:00  812676         67    NJ
1  2019-11-01 01:18:00  812761         65    NJ
2  2019-11-01 01:21:00  812669         56    NY
3  2019-11-01 01:53:00  812612         53    NY
4  2019-11-01 12:02:00  812588         59    DE
wxdata_nov2019e.csv saved
The key 'storm-report' is present.

The key 'storm-report' is present.

                  Date      ID  Magnitude State
0  2019-11-19 13:29:00  817269         44    AZ
1  2019-11-19 13:35:00  817271         44    AZ
2  2019-11-20 11:50:00  817323         46    CA
3  2019-11-20 12:15:00  818510         47    CA
4  2019-11-26 22:35:00  819638         60    IA
wxdata_nov2019l.csv saved


In [14]:
dec2019_1_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2019-12-01T00:00:00&enddate=2019-12-16T00:00:00&event=tstm+wnd+gst"
dec2019_1_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2019-12-01T00:00:00&enddate=2019-12-16T00:00:00&event=marine+tstm+wind"
dec2019_2_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2019-12-16T00:00:00&enddate=2020-01-01T00:00:00&event=tstm+wnd+gst"
dec2019_2_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2019-12-16T00:00:00&enddate=2020-01-01T00:00:00&event=marine+tstm+wind"

df_wxdata_dec2019e_twg = dataframe_strmrpt(dec2019_1_twg)
df_wxdata_dec2019e_mtw = dataframe_strmrpt(dec2019_1_mtw)
df_wxdata_dec2019e = concat_wxdf(df_wxdata_dec2019e_twg,df_wxdata_dec2019e_mtw)
df_wxdata_dec2019e.to_csv(r'data\wxdata_dec2019e.csv')
print("wxdata_dec2019e.csv saved")

df_wxdata_dec2019l_twg = dataframe_strmrpt(dec2019_2_twg)
df_wxdata_dec2019l_mtw = dataframe_strmrpt(dec2019_2_mtw)
df_wxdata_dec2019l = concat_wxdf(df_wxdata_dec2019l_twg,df_wxdata_dec2019l_mtw)
df_wxdata_dec2019l.to_csv(r'data\wxdata_dec2019l.csv')
print("wxdata_dec20191.csv saved")

The key 'storm-report' is present.

The key 'storm-report' is present.

                  Date      ID  Magnitude State
0  2019-12-14 03:24:00  829460         43    FL
1  2019-12-14 04:25:00  829464          0    FL
2  2019-12-14 04:25:00  830518          0    FL
3  2019-12-14 05:58:00  829477         43    FL
4  2019-12-14 06:55:00  829474         39    FL
wxdata_dec2019e.csv saved
The key 'storm-report' is present.

The key 'storm-report' is present.

                  Date      ID  Magnitude State
0  2019-12-16 17:10:00  831600         54    TN
1  2019-12-16 17:52:00  831829         62    AL
2  2019-12-16 17:57:00  833654         70    AL
3  2019-12-16 18:15:00  832986         47    TN
4  2019-12-16 19:13:00  831680         60    MS
wxdata_dec20191.csv saved


In [15]:
jan2020_1_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-01-01T00:00:00&enddate=2020-01-16T00:00:00&event=tstm+wnd+gst"
jan2020_1_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-01-01T00:00:00&enddate=2020-01-16T00:00:00&event=marine+tstm+wind"
jan2020_2_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-01-16T00:00:00&enddate=2020-02-01T00:00:00&event=marine+tstm+wind"

df_wxdata_jan2020e_twg = dataframe_strmrpt(jan2020_1_twg)
df_wxdata_jan2020e_mtw = dataframe_strmrpt(jan2020_1_mtw)
df_wxdata_jan2020e = concat_wxdf(df_wxdata_jan2020e_twg,df_wxdata_jan2020e_mtw)
df_wxdata_jan2020e.to_csv(r'data\wxdata_jan2020e.csv')
print("wxdata_jan2020e.csv saved")

df_wxdata_jan2020l = dataframe_strmrpt(jan2020_2_mtw)
df_wxdata_jan2020l.to_csv(r'data\wxdata_jan2020l.csv')
print("wxdata_jan2020l.csv saved")

The key 'storm-report' is present.

The key 'storm-report' is present.

                  Date      ID  Magnitude State
0  2020-01-04 12:55:00  839001         39    FL
1  2020-01-04 13:22:00  839013         42    FL
2  2020-01-04 13:30:00  839012         39    FL
3  2020-01-04 14:18:00  839016         41    FL
4  2020-01-04 14:34:00  839020         44    FL
wxdata_jan2020e.csv saved
The key 'storm-report' is present.

wxdata_jan2020l.csv saved


In [16]:
feb2020_1_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-02-01T00:00:00&enddate=2020-02-15T00:00:00&event=tstm+wnd+gst"
feb2020_1_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-02-01T00:00:00&enddate=2020-02-15T00:00:00&event=marine+tstm+wind"
feb2020_2_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-02-15T00:00:00&enddate=2020-03-01T00:00:00&event=tstm+wnd+gst"
feb2020_2_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-02-15T00:00:00&enddate=2020-03-01T00:00:00&event=marine+tstm+wind"

df_wxdata_feb2020e_twg = dataframe_strmrpt(feb2020_1_twg)
df_wxdata_feb2020e_mtw = dataframe_strmrpt(feb2020_1_mtw)
df_wxdata_feb2020e = concat_wxdf(df_wxdata_feb2020e_twg,df_wxdata_feb2020e_mtw)
df_wxdata_feb2020e.to_csv(r'data\wxdata_feb2020e.csv')
print("wxdata_feb2020e.csv saved")

df_wxdata_feb2020l_twg = dataframe_strmrpt(feb2020_2_twg)
df_wxdata_feb2020l_mtw = dataframe_strmrpt(feb2020_2_mtw)
df_wxdata_feb2020l = concat_wxdf(df_wxdata_feb2020l_twg,df_wxdata_feb2020l_mtw)
df_wxdata_feb2020l.to_csv(r'data\wxdata_feb2020l.csv')
print("wxdata_feb2020l.csv saved")


The key 'storm-report' is present.

The key 'storm-report' is present.

                  Date      ID  Magnitude State
0  2020-02-05 13:40:00  854194         56    LA
1  2020-02-05 18:01:00  854316         61    TN
2  2020-02-05 18:51:00  854403         58    TN
3  2020-02-06 12:25:00  856237         45    FL
4  2020-02-06 12:25:00  857913         45    FL
wxdata_feb2020e.csv saved
The key 'storm-report' is present.

The key 'storm-report' is present.

                  Date      ID  Magnitude State
0  2020-02-21 05:00:00  863018         41    FL
1  2020-02-23 15:54:00  863192         60    TX
2  2020-02-24 18:52:00  863760         60    TX
3  2020-02-24 19:35:00  863443         59    TX
4  2020-02-26 15:46:00  864150         41    FL
wxdata_feb2020l.csv saved


In [17]:
mar2020_1_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-03-01T00:00:00&enddate=2020-03-16T00:00:00&event=tstm+wnd+gst"
mar2020_1_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-03-01T00:00:00&enddate=2020-03-16T00:00:00&event=marine+tstm+wind"
mar2020_2_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-03-16T00:00:00&enddate=2020-04-01T00:00:00&event=tstm+wnd+gst"
mar2020_2_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-03-16T00:00:00&enddate=2020-04-01T00:00:00&event=marine+tstm+wind"

df_wxdata_mar2020e_twg = dataframe_strmrpt(mar2020_1_twg)
df_wxdata_mar2020e_mtw = dataframe_strmrpt(mar2020_1_mtw)
df_wxdata_mar2020e = concat_wxdf(df_wxdata_mar2020e_twg,df_wxdata_mar2020e_mtw)
df_wxdata_mar2020e.to_csv(r'data\wxdata_mar2020e.csv')
print("wxdata_mar2020e.csv saved")

df_wxdata_mar2020l_twg = dataframe_strmrpt(mar2020_2_twg)
df_wxdata_mar2020l_mtw = dataframe_strmrpt(mar2020_2_mtw)
df_wxdata_mar2020l = concat_wxdf(df_wxdata_mar2020l_twg,df_wxdata_mar2020l_mtw)
df_wxdata_mar2020l.to_csv(r'data\wxdata_mar2020l.csv')
print("wxdata_mar2020l.csv saved")


The key 'storm-report' is present.

The key 'storm-report' is present.

                  Date      ID  Magnitude State
0  2020-03-03 01:19:00  866047         60    TN
1  2020-03-03 01:19:00  867446         60    TN
2  2020-03-03 17:37:00  866310         53    OH
3  2020-03-03 18:22:00  866312         45    OH
4  2020-03-03 18:40:00  866323         46    OH
wxdata_mar2020e.csv saved
The key 'storm-report' is present.

The key 'storm-report' is present.

                  Date      ID  Magnitude State
0  2020-03-18 14:55:00  869496         47    AZ
1  2020-03-18 15:53:00  869471         62    AZ
2  2020-03-18 17:55:00  869475         58    TX
3  2020-03-18 18:45:00  869503         57    AZ
4  2020-03-18 18:47:00  869500         51    AZ
wxdata_mar2020l.csv saved


In [19]:
apr2020_1_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-04-01T00:00:00&enddate=2020-04-16T00:00:00&event=tstm+wnd+gst"
apr2020_1_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-04-01T00:00:00&enddate=2020-04-16T00:00:00&event=marine+tstm+wind"
apr2020_2_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-04-16T00:00:00&enddate=2020-04-30T23:00:00&event=tstm+wnd+gst"
apr2020_2_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-04-16T00:00:00&enddate=2020-04-30T23:00:00&event=marine+tstm+wind"

df_wxdata_apr2020e_twg = dataframe_strmrpt(apr2020_1_twg)
df_wxdata_apr2020e_mtw = dataframe_strmrpt(apr2020_1_mtw)
df_wxdata_apr2020e = concat_wxdf(df_wxdata_apr2020e_twg,df_wxdata_apr2020e_mtw)
df_wxdata_apr2020e.to_csv(r'data\wxdata_apr2020e.csv')
print("wxdata_apr2020e.csv saved")

df_wxdata_apr2020l_twg = dataframe_strmrpt(apr2020_2_twg)
df_wxdata_apr2020l_mtw = dataframe_strmrpt(apr2020_2_mtw)
df_wxdata_apr2020l = concat_wxdf(df_wxdata_apr2020l_twg,df_wxdata_apr2020l_mtw)
df_wxdata_apr2020l.to_csv(r'data\wxdata_apr2020l.csv')
print("wxdata_apr2020l.csv saved")

The key 'storm-report' is present.

The key 'storm-report' is present.

                  Date      ID  Magnitude State
0  2020-04-03 17:57:00  874756         60    TX
1  2020-04-05 19:13:00  874953         56    WY
2  2020-04-07 16:15:00  875129         58    MD
3  2020-04-07 19:10:00  875507         60    VA
4  2020-04-07 22:18:00  875320         63    MI
wxdata_apr2020e.csv saved
The key 'storm-report' is present.

The key 'storm-report' is present.

                  Date      ID  Magnitude State
0  2020-04-18 11:06:00  883680         44    FL
1  2020-04-18 11:19:00  883679         44    FL
2  2020-04-18 13:06:00  883499         41    FL
3  2020-04-18 13:06:00  883585         41    FL
4  2020-04-18 13:21:00  883586         43    FL
wxdata_apr2020l.csv saved


In [20]:
#Invoke requests to call the WxData Storm Reports API Builder. Execute an API call for each semi-monthly period between November 2019 and April 2020.
#See https://wxdata.com/storm-reports-api-format#event
#nov2020_1_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-11-01T00:00:00&enddate=2020-11-16T00:00:00&event=tstm+wnd+gst"
nov2020_1_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-11-01T00:00:00&enddate=2020-11-16T00:00:00&event=marine+tstm+wind"
nov2020_2_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-11-16T00:00:00&enddate=2020-12-01T00:00:00&event=tstm+wnd+gst"
nov2020_2_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-11-16T00:00:00&enddate=2020-12-01T00:00:00&event=tstm+wnd+gst"

#df_wxdata_nov2020e_twg = dataframe_strmrpt(nov2020_1_twg)
#df_wxdata_nov2020e_twg.to_csv(r'data\wxdata_nov2020e_twg.csv')
#print("wxdata_nov2020e_twg.csv saved")
df_wxdata_nov2020e = dataframe_strmrpt(nov2020_1_mtw)
df_wxdata_nov2020e.to_csv(r'data\wxdata_nov2020e.csv')
print("wxdata_nov2020e.csv saved")

df_wxdata_nov2020l_twg = dataframe_strmrpt(nov2020_2_twg)
df_wxdata_nov2020l_mtw = dataframe_strmrpt(nov2020_2_mtw)
df_wxdata_nov2020l = concat_wxdf(df_wxdata_nov2020l_twg,df_wxdata_nov2020l_mtw)
df_wxdata_nov2020l.to_csv(r'data\wxdata_nov2020l.csv')
print("wxdata_nov2020l.csv saved")


The key 'storm-report' is present.

wxdata_nov2020e.csv saved
The key 'storm-report' is present.

The key 'storm-report' is present.

                  Date      ID  Magnitude State
0  2020-11-23 11:50:00  951253         63    MA
1  2020-11-23 12:16:00  951254         61    MA
2  2020-11-23 12:19:00  951255         65    MA
3  2020-11-24 14:53:00  951737         64    OK
4  2020-11-24 15:25:00  951743         58    OK
wxdata_nov2020l.csv saved
